In [39]:
import sys
import pandas as pd
variants = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KHAIGPRX981_final_DP.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
variants.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
df = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/13genes_coordinates_haplotypes_07122023.xlsx", header = 0)
# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    gene = row['Gene']  # Assuming 'Gene' is the name of your gene column
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos, gene))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end, gene in ranges:
            if start <= pos <= end:
                return 'Covered', start, end, gene
    return 'Not_Covered', None, None, None  # Return None for start, end, and gene if not covered

# Step 3: Apply the function to create the new columns in data
variants['Covered/Not_Covered'], variants['Start_Pos_Covered'], variants['End_Pos_Covered'], variants['Gene'] = zip(*variants.apply(check_coverage, axis=1))
# Step 4: Create new columns for Covered rows
covered_rows = variants['Covered/Not_Covered'] == 'Covered'
variants.loc[covered_rows, 'Covered_Chromosome'] = variants.loc[covered_rows, 'CHROM']
variants.loc[covered_rows, 'Covered_Start_Pos'] = variants.loc[covered_rows, 'Start_Pos_Covered']
variants.loc[covered_rows, 'Covered_End_Pos'] = variants.loc[covered_rows, 'End_Pos_Covered']
variants.loc[covered_rows, 'Gene'] = variants.loc[covered_rows, 'Gene']

# Drop temporary columns
variants.drop(['Start_Pos_Covered', 'End_Pos_Covered'], axis=1, inplace=True)

# Display the DataFrame
variants = variants[variants['Covered/Not_Covered'] == 'Covered']
df1 = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/main_new_data.xlsx")
df_x = df1[['CHROM', 'POS', 'REF', 'ALT', 'Haplotype', 'Type']]
df_x = df_x.groupby(['CHROM', 'POS', 'REF', 'ALT']).agg({'Haplotype': lambda x: ','.join(x.unique()), 
                                                        'Type' : 'first'}).reset_index()

merged_df = pd.merge(variants, df_x, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort = False)
merged_df['Type'].fillna('Snp', inplace=True)
merged_df['Haplotype'].fillna('Snp', inplace=True)
df2 = df1.copy()
df2['Gene'] = df2['Haplotype'].str.split('*').str.get(0)
df2 = df2[['CHROM', 'POS', 'Gene']]
df2 = df2.drop_duplicates(subset=['CHROM', 'POS', 'Gene'])
def find_nearest_pos(row):
    df2_subset = df2[(df2['Gene'] == row['Gene']) & (df2['CHROM'] == row['CHROM'])]
    if not df2_subset.empty:
        nearest_pos_index = (df2_subset['POS'] - row['POS']).abs().idxmin()
        nearest_pos = df2_subset.loc[nearest_pos_index, 'POS']
        return nearest_pos
    else:
        return None

# Apply the function to create 'POS_df2' column in data
merged_df['POS_df2'] = merged_df.apply(find_nearest_pos, axis=1)
merged_df['POS_diff'] = merged_df['POS_df2'] - merged_df['POS']
merged_df['CHROM_df2'] = merged_df['Covered_Chromosome']
df3 = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/main_new_data.xlsx")
df3 = df3.groupby(['CHROM', 'POS']).agg({'Haplotype': lambda x: ','.join(x.unique())}).reset_index()
df3.rename(columns={'POS':'POS_df2', 'Haplotype':'Haplotype_updated'}, inplace=True)
final = pd.merge(merged_df, df3, on = ['CHROM', 'POS_df2'], how = 'left', sort = False)
final['Haplotype_updated'] = final['Haplotype_updated'].apply(lambda x: ','.join(sorted(set(x.split(',')))))
final

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_df2,POS_diff,CHROM_df2,Haplotype_updated
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,233760498.0,-3495.0,chr2,UGT1A1*6
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99652771.0,4480.0,chr7,CYP3A5*7
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99665212.0,-3483.0,chr7,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,99764003.0,160.0,chr7,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,18400285.0,0.0,chr8,"NAT2*13A,NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7..."
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,18400593.0,0.0,chr8,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O"
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,...,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,94762856.0,52.0,chr10,CYP2C19*19
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,94775367.0,0.0,chr10,CYP2C19*2
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21178615.0,-50.0,chr12,"SLCO1B1*15,SLCO1B1*5"
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21205999.0,-33629.0,chr12,SLCO1B1*9


In [1]:
import sys
import pandas as pd
variants = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KHAIGPRX981_final_DP.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
variants.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
variants

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,chr1,11786390,rs4845884,G,A,.,PASS,"ADP=35;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06669,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:200:35:35:0:35:100%:8.9137E-21:0:50:0:0:28:7
1,chr1,11787379,rs2184226,T,C,.,PASS,"ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.9403,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:39:24:24:13:11:45.83%:1.1047E-4:53:61:13:0...
2,chr1,11787702,rs3737966,C,T,.,PASS,"ADP=21;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.4415,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:40:21:21:10:11:52.38%:8.2399E-5:62:46:9:1:7:4
3,chr1,11788822,rs2077360,A,G,.,PASS,"ADP=21;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06949,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:117:21:21:0:21:100%:1.8578E-12:0:54:0:0:12:9
4,chr1,11789390,rs868014,A,G,.,PASS,"ADP=39;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06709,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:224:39:39:0:39:100%:3.6742E-23:0:60:0:0:32:7
...,...,...,...,...,...,...,...,...,...,...
112,chr21,45531642,rs12659,A,G,.,PASS,"ADP=61;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.4469,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:130:61:61:27:34:55.74%:8.7556E-14:52:53:17...
113,chr21,45537880,rs1051266,T,C,.,PASS,"ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5114,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:138:58:58:23:35:60.34%:1.5534E-14:64:68:20...
114,chrX,154531643,rs1050757,C,T,.,PASS,"ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5846,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:105:19:19:0:19:100%:2.8292E-11:0:54:0:0:14:5
115,chrX,154532293,rs2071429,G,A,.,PASS,"ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5852,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:105:20:19:0:19:100%:2.8292E-11:0:59:0:0:16:3


In [2]:
df = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/13genes_coordinates_haplotypes_07122023.xlsx", header = 0)
df

,Gene,chromosome,Extended_Start_pos,Extended_End_pos
0,CYP2A6,chr19,40843541,40850447
1,CYP2B6,chr19,40991282,41018398
2,CYP2C19,chr10,94762681,94855547
3,CYP2C9,chr10,94938658,94990091
4,CYP2D6,chr22,42126499,42130810
5,CYP3A4,chr7,99756967,99784184
6,CYP3A5,chr7,99648194,99679996
7,NAT2,chr8,18391282,18401218
8,NUDT15,chr13,48037726,48047221
9,SLCO1B1,chr12,21131194,21239796


In [3]:
# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    gene = row['Gene']  # Assuming 'Gene' is the name of your gene column
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos, gene))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end, gene in ranges:
            if start <= pos <= end:
                return 'Covered', start, end, gene
    return 'Not_Covered', None, None, None  # Return None for start, end, and gene if not covered

# Step 3: Apply the function to create the new columns in data
variants['Covered/Not_Covered'], variants['Start_Pos_Covered'], variants['End_Pos_Covered'], variants['Gene'] = zip(*variants.apply(check_coverage, axis=1))
variants

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Covered/Not_Covered,Start_Pos_Covered,End_Pos_Covered,Gene
0,chr1,11786390,rs4845884,G,A,.,PASS,"ADP=35;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06669,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:200:35:35:0:35:100%:8.9137E-21:0:50:0:0:28:7,Not_Covered,NaN,NaN,None
1,chr1,11787379,rs2184226,T,C,.,PASS,"ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.9403,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:39:24:24:13:11:45.83%:1.1047E-4:53:61:13:0...,Not_Covered,NaN,NaN,None
2,chr1,11787702,rs3737966,C,T,.,PASS,"ADP=21;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.4415,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:40:21:21:10:11:52.38%:8.2399E-5:62:46:9:1:7:4,Not_Covered,NaN,NaN,None
3,chr1,11788822,rs2077360,A,G,.,PASS,"ADP=21;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06949,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:117:21:21:0:21:100%:1.8578E-12:0:54:0:0:12:9,Not_Covered,NaN,NaN,None
4,chr1,11789390,rs868014,A,G,.,PASS,"ADP=39;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.06709,0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:224:39:39:0:39:100%:3.6742E-23:0:60:0:0:32:7,Not_Covered,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,chr21,45531642,rs12659,A,G,.,PASS,"ADP=61;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.4469,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:130:61:61:27:34:55.74%:8.7556E-14:52:53:17...,Not_Covered,NaN,NaN,None
113,chr21,45537880,rs1051266,T,C,.,PASS,"ADP=58;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5114,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:138:58:58:23:35:60.34%:1.5534E-14:64:68:20...,Not_Covered,NaN,NaN,None
114,chrX,154531643,rs1050757,C,T,.,PASS,"ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5846,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:105:19:19:0:19:100%:2.8292E-11:0:54:0:0:14:5,Covered,154531391.0,154546846.0,G6PD
115,chrX,154532293,rs2071429,G,A,.,PASS,"ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5852,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:105:20:19:0:19:100%:2.8292E-11:0:59:0:0:16:3,Covered,154531391.0,154546846.0,G6PD


In [4]:
# Step 4: Create new columns for Covered rows
covered_rows = variants['Covered/Not_Covered'] == 'Covered'
variants.loc[covered_rows, 'Covered_Chromosome'] = variants.loc[covered_rows, 'CHROM']
variants.loc[covered_rows, 'Covered_Start_Pos'] = variants.loc[covered_rows, 'Start_Pos_Covered']
variants.loc[covered_rows, 'Covered_End_Pos'] = variants.loc[covered_rows, 'End_Pos_Covered']
variants.loc[covered_rows, 'Gene'] = variants.loc[covered_rows, 'Gene']

# Drop temporary columns
variants.drop(['Start_Pos_Covered', 'End_Pos_Covered'], axis=1, inplace=True)

# Display the DataFrame
variants = variants[variants['Covered/Not_Covered'] == 'Covered']
variants

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Covered/Not_Covered,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos
36,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,Covered,UGT1A1,chr2,233760270.0,233773300.0
54,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0
55,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0
56,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,Covered,CYP3A4,chr7,99756967.0,99784184.0
58,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0
59,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,Covered,NAT2,chr8,18391282.0,18401218.0
60,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,Covered,CYP2C19,chr10,94762681.0,94855547.0
61,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,Covered,CYP2C19,chr10,94762681.0,94855547.0
62,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,Covered,SLCO1B1,chr12,21131194.0,21239796.0
63,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,Covered,SLCO1B1,chr12,21131194.0,21239796.0


In [9]:
df1 = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/main_new_data.xlsx")
df_x = df1[['CHROM', 'POS', 'REF', 'ALT', 'Haplotype', 'Type']]
df_x = df_x.groupby(['CHROM', 'POS', 'REF', 'ALT']).agg({'Haplotype': lambda x: ','.join(x.unique()), 
                                                        'Type' : 'first'}).reset_index()

merged_df = pd.merge(variants, df_x, on=['CHROM', 'POS', 'REF', 'ALT'], how='left', sort = False)
merged_df['Type'].fillna('Snp', inplace=True)
merged_df['Haplotype'].fillna('Snp', inplace=True)
merged_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Covered/Not_Covered,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,Covered,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,Covered,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,Covered,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,Covered,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,Covered,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,Covered,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,Covered,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp


In [10]:
df2 = df1.copy()
df2['Gene'] = df2['Haplotype'].str.split('*').str.get(0)
df2 = df2[['CHROM', 'POS', 'Gene']]
df2 = df2.drop_duplicates(subset=['CHROM', 'POS', 'Gene'])
df2

,CHROM,POS,Gene
0,chr19,40848628.0,CYP2A6
1,chr19,40843869.0,CYP2A6
2,chr19,40843749.0,CYP2A6
3,chr19,40843746.0,CYP2A6
4,chr19,40843742.0,CYP2A6
...,...,...,...
723,chrX,154535330.0,G6PD
725,chrX,154536151.0,G6PD
727,chrX,154546116.0,G6PD
728,chrX,154533004.0,G6PD


In [11]:
def find_nearest_pos(row):
    df2_subset = df2[(df2['Gene'] == row['Gene']) & (df2['CHROM'] == row['CHROM'])]
    if not df2_subset.empty:
        nearest_pos_index = (df2_subset['POS'] - row['POS']).abs().idxmin()
        nearest_pos = df2_subset.loc[nearest_pos_index, 'POS']
        return nearest_pos
    else:
        return None

# Apply the function to create 'POS_df2' column in data
merged_df['POS_df2'] = merged_df.apply(find_nearest_pos, axis=1)
merged_df['POS_diff'] = merged_df['POS_df2'] - merged_df['POS']
merged_df['CHROM_df2'] = merged_df['Covered_Chromosome']
merged_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Covered/Not_Covered,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_df2,POS_diff,CHROM_df2
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,Covered,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,233760498.0,-3495.0,chr2
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99652771.0,4480.0,chr7
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99665212.0,-3483.0,chr7
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,Covered,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,99764003.0,160.0,chr7
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,18400285.0,0.0,chr8
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,Covered,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,18400593.0,0.0,chr8
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,Covered,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,94762856.0,52.0,chr10
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,Covered,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,94775367.0,0.0,chr10
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,Covered,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21178615.0,-50.0,chr12
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,Covered,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21205999.0,-33629.0,chr12


In [36]:
df3 = pd.read_excel(r"C:/Users/GenepoweRx_Madhu/Downloads/main_new_data.xlsx")
df3 = df3.groupby(['CHROM', 'POS']).agg({'Haplotype': lambda x: ','.join(x.unique())}).reset_index()
df3.rename(columns={'POS':'POS_df2', 'Haplotype':'Haplotype_updated'}, inplace=True)
df3

,CHROM,POS_df2,Haplotype_updated
0,chr10,94761900.0,CYP2C19*17
1,chr10,94762706.0,CYP2C19*4
2,chr10,94762856.0,CYP2C19*19
3,chr10,94775367.0,CYP2C19*2
4,chr10,94775416.0,CYP2C19*8
...,...,...,...
392,chrX,154536224.0,G6PD*ClinVar
393,chrX,154546052.0,G6PD*Sunderland
394,chrX,154546061.0,G6PD*Gaohe
395,chrX,154546116.0,G6PD*Lages


In [37]:
final = pd.merge(merged_df, df3, on = ['CHROM', 'POS_df2'], how = 'left', sort = False)
final

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_df2,POS_diff,CHROM_df2,Haplotype_updated
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,233760498.0,-3495.0,chr2,UGT1A1*6
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99652771.0,4480.0,chr7,CYP3A5*7
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99665212.0,-3483.0,chr7,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,99764003.0,160.0,chr7,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,18400285.0,0.0,chr8,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G..."
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,18400593.0,0.0,chr8,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A..."
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,...,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,94762856.0,52.0,chr10,CYP2C19*19
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,94775367.0,0.0,chr10,CYP2C19*2
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21178615.0,-50.0,chr12,"SLCO1B1*5,SLCO1B1*15"
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21205999.0,-33629.0,chr12,SLCO1B1*9


In [38]:
final['Haplotype_updated'] = final['Haplotype_updated'].apply(lambda x: ','.join(sorted(set(x.split(',')))))
final

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_df2,POS_diff,CHROM_df2,Haplotype_updated
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,233760498.0,-3495.0,chr2,UGT1A1*6
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99652771.0,4480.0,chr7,CYP3A5*7
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,99665212.0,-3483.0,chr7,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,99764003.0,160.0,chr7,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,18400285.0,0.0,chr8,"NAT2*13A,NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7..."
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,18400593.0,0.0,chr8,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O"
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,...,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,94762856.0,52.0,chr10,CYP2C19*19
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,94775367.0,0.0,chr10,CYP2C19*2
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21178615.0,-50.0,chr12,"SLCO1B1*15,SLCO1B1*5"
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,21205999.0,-33629.0,chr12,SLCO1B1*9


In [6]:
df2 = df1.copy()
df2['Gene'] = df2['Haplotype'].str.split('*').str.get(0)
df2 = df2[['CHROM', 'POS', 'Gene']]
df2 = df2.drop_duplicates(subset=['CHROM', 'POS', 'Gene'])
#df2 = df2.rename(columns={'Haplotype': 'Haplotype_mapped'})
#df2 = df2.groupby(['CHROM', 'POS', 'Gene']).agg({'Haplotype_mapped': lambda x: ','.join(x.unique())}).reset_index()
df2

,CHROM,POS,Gene,Haplotype_mapped
0,chr10,94761900.0,CYP2C19,CYP2C19*17
1,chr10,94762706.0,CYP2C19,CYP2C19*4
2,chr10,94762856.0,CYP2C19,CYP2C19*19
3,chr10,94775367.0,CYP2C19,CYP2C19*2
4,chr10,94775416.0,CYP2C19,CYP2C19*8
...,...,...,...,...
392,chrX,154536224.0,G6PD,G6PD*ClinVar
393,chrX,154546052.0,G6PD,G6PD*Sunderland
394,chrX,154546061.0,G6PD,G6PD*Gaohe
395,chrX,154546116.0,G6PD,G6PD*Lages


In [19]:
# Merge datasets on the 'Gene' column
merged_df_new = pd.merge(merged_df, df2, on='Gene', suffixes=('_merged_df', '_df2'))

# Calculate the absolute difference between 'POS' values
merged_df_new['POS_Difference'] = abs(merged_df_new['POS_merged_df'] - merged_df_new['POS_df2'])
merged_df_new

,CHROM_merged_df,POS_merged_df,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,CHROM_df2,POS_df2,Haplotype_mapped,POS_Difference
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,chr2,233760234.0,UGT1A1*28,3759.0
1,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,chr2,233760498.0,UGT1A1*6,3495.0
2,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,chr7,99652771.0,CYP3A5*7,4480.0
3,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,chr7,99665212.0,CYP3A5*6,16921.0
4,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,chr7,99672916.0,CYP3A5*3,24625.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,chrX,154532439,rs2230037,A,G,.,PASS,"ADP=29;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7841,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:164:29:29:0:29:100%:3.3259E-17:0:50:0:0:22:7,...,G6PD,chrX,154531391.0,154546846.0,G6PD*ClinVar,Haplo,chrX,154536224.0,G6PD*ClinVar,3785.0
979,chrX,154532439,rs2230037,A,G,.,PASS,"ADP=29;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7841,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:164:29:29:0:29:100%:3.3259E-17:0:50:0:0:22:7,...,G6PD,chrX,154531391.0,154546846.0,G6PD*ClinVar,Haplo,chrX,154546052.0,G6PD*Sunderland,13613.0
980,chrX,154532439,rs2230037,A,G,.,PASS,"ADP=29;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7841,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:164:29:29:0:29:100%:3.3259E-17:0:50:0:0:22:7,...,G6PD,chrX,154531391.0,154546846.0,G6PD*ClinVar,Haplo,chrX,154546061.0,G6PD*Gaohe,13622.0
981,chrX,154532439,rs2230037,A,G,.,PASS,"ADP=29;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7841,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:164:29:29:0:29:100%:3.3259E-17:0:50:0:0:22:7,...,G6PD,chrX,154531391.0,154546846.0,G6PD*ClinVar,Haplo,chrX,154546116.0,G6PD*Lages,13677.0


In [20]:
min_diff_df = merged_df_new.loc[merged_df_new.groupby('Gene')['POS_Difference'].idxmin()]
min_diff_df

,CHROM_merged_df,POS_merged_df,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,CHROM_df2,POS_df2,Haplotype_mapped,POS_Difference
238,chr19,40850376,rs1137115,T,C,.,PASS,"ADP=62;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.2394,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:62:62:0:62:100%:6.5755E-37:0:67:0:0:53:9,...,CYP2A6,chr19,40843541.0,40850447.0,Snp,Snp,chr19,40850341.0,CYP2A6*14,35.0
244,chr19,41006936,rs3745274,G,T,.,PASS,ADP=22;WT=0;HET=1;HOM=0;NC=0;ASP;ASS;CAF=0.684...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:35:22:22:12:10:45.45%:2.6061E-4:61:55:11:1...,...,CYP2B6,chr19,40991282.0,41018398.0,"CYP2B6*6,CYP2B6*9,CYP2B6*26,CYP2B6*6,CYP2B6*26",Haplo,chr19,41006936.0,"CYP2B6*6,CYP2B6*9,CYP2B6*26,CYP2B6*6,CYP2B6*26",0.0
105,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,chr10,94775367.0,CYP2C19*2,0.0
10,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,chr7,99764003.0,CYP3A4*18,160.0
6,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,chr7,99665212.0,CYP3A5*6,3483.0
285,chrX,154531643,rs1050757,C,T,.,PASS,"ADP=19;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5846,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:105:19:19:0:19:100%:2.8292E-11:0:54:0:0:14:5,...,G6PD,chrX,154531391.0,154546846.0,G6PD*ClinVar,Haplo,chrX,154531643.0,G6PD*ClinVar,0.0
21,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,chr8,18400285.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",0.0
118,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,chr12,21178615.0,"SLCO1B1*5,SLCO1B1*15",50.0
1,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,chr2,233760498.0,UGT1A1*6,3495.0


In [21]:
# Merge the minimum difference information back to df1
result_df = pd.merge(merged_df, min_diff_df[['Gene', 'POS_Difference', 'CHROM_df2', 'POS_df2']], on='Gene', how='left')

result_df['Haplotype_mapped'] = min_diff_df.set_index(['Gene', 'CHROM_df2', 'POS_df2']).loc[result_df.set_index(['Gene', 'CHROM_df2', 'POS_df2']).index, 'Haplotype_mapped'].values
result_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_Difference,CHROM_df2,POS_df2,Haplotype_mapped
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,3495.0,chr2,233760498.0,UGT1A1*6
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,3483.0,chr7,99665212.0,CYP3A5*6
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,3483.0,chr7,99665212.0,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,160.0,chr7,99764003.0,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,0.0,chr8,18400285.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G..."
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,0.0,chr8,18400285.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G..."
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,...,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,0.0,chr10,94775367.0,CYP2C19*2
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,0.0,chr10,94775367.0,CYP2C19*2
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,50.0,chr12,21178615.0,"SLCO1B1*5,SLCO1B1*15"
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,50.0,chr12,21178615.0,"SLCO1B1*5,SLCO1B1*15"


In [16]:
# Merge datasets on the 'Gene' column
merged_df_new = pd.merge(merged_df, df2, on='Gene', suffixes=('_merged_df', '_df2'))

# Calculate the absolute difference between 'POS' values
merged_df_new['POS_Difference'] = abs(merged_df_new['POS_merged_df'] - merged_df_new['POS_df2'])

# Find the row with the minimum absolute difference for each gene
min_diff_df = merged_df_new.loc[merged_df_new.groupby('Gene')['POS_Difference'].idxmin()]

# Merge the minimum difference information back to df1
result_df = pd.merge(merged_df, min_diff_df[['Gene', 'POS_Difference', 'CHROM_df2', 'POS_df2']], on='Gene', how='left')

result_df['Haplotype_mapped'] = min_diff_df.set_index(['Gene', 'CHROM_df2', 'POS_df2']).loc[result_df.set_index(['Gene', 'CHROM_df2', 'POS_df2']).index, 'Haplotype_mapped'].values

# Display the result or save it to a new dataset
result_df['POS_Difference'] = result_df['POS'] - result_df['POS_df2']
mask = result_df['Haplotype'] != 'Snp'
result_df.loc[mask, 'POS_Difference'] = None
result_df.loc[mask, 'CHROM_df2'] = result_df.loc[mask, 'CHROM']
result_df.loc[mask, 'POS_df2'] = result_df.loc[mask, 'POS']
result_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,Type,POS_Difference,CHROM_df2,POS_df2,Haplotype_mapped
0,chr2,233763993,rs6742078,G,T,.,PASS,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,...,UGT1A1,chr2,233760270.0,233773300.0,Snp,Snp,3495.0,chr2,233760498.0,UGT1A1*6
1,chr7,99648291,rs15524,A,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,-16921.0,chr7,99665212.0,CYP3A5*6
2,chr7,99668695,rs4646450,G,A,.,PASS,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,...,CYP3A5,chr7,99648194.0,99679996.0,Snp,Snp,3483.0,chr7,99665212.0,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,.,PASS,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,...,CYP3A4,chr7,99756967.0,99784184.0,Snp,Snp,-160.0,chr7,99764003.0,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,.,PASS,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G...",Haplo,NaN,chr8,18400285.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G..."
5,chr8,18400593,rs1799930,G,A,.,PASS,"ADP=35;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.735,0.2...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:57:35:35:19:16:45.71%:1.637E-6:60:54:15:4:...,...,NAT2,chr8,18391282.0,18401218.0,"NAT2*4,NAT2*6A,NAT2*6B,NAT2*6J,NAT2*6O,NAT2*6A...",Haplo,NaN,chr8,18400593.0,"NAT2*4,NAT2*6A,NAT2*6J,NAT2*6O,NAT2*7B,NAT2*7G..."
6,chr10,94762804,rs17885098,C,T,.,PASS,"ADP=36;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9081,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:206:36:36:0:36:100%:2.2598E-21:0:61:0:0:31:5,...,CYP2C19,chr10,94762681.0,94855547.0,Snp,Snp,-12563.0,chr10,94775367.0,CYP2C19*2
7,chr10,94775367,rs12769205,A,G,.,PASS,"ADP=51;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.7716,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:51:51:1:50:98.04%:1.3013E-28:74:51:1:0...,...,CYP2C19,chr10,94762681.0,94855547.0,CYP2C19*2,Haplo,NaN,chr10,94775367.0,CYP2C19*2
8,chr12,21178665,rs4149057,T,C,.,PASS,"ADP=33;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6326,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:73:33:33:14:19:57.58%:4.7323E-8:53:58:9:5:...,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,50.0,chr12,21178615.0,"SLCO1B1*5,SLCO1B1*15"
9,chr12,21239628,rs4149087,T,G,.,PASS,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5523,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:24:16:16:9:7:43.75%:3.3988E-3:57:50:8:1:3:4,...,SLCO1B1,chr12,21131194.0,21239796.0,Snp,Snp,61013.0,chr12,21178615.0,"SLCO1B1*5,SLCO1B1*15"


In [13]:
result_df = result_df.drop(['QUAL', 'FILTER', 'Type'], axis=1)
result_df['Haplotype_mapped'] = result_df['Haplotype_mapped'].apply(lambda x: ','.join(set(x.split(','))))
result_df

,CHROM,POS,rsID,REF,ALT,INFO,FORMAT,SAMPLE,Covered/Not_Covered,Gene,Covered_Chromosome,Covered_Start_Pos,Covered_End_Pos,Haplotype,POS_Difference,CHROM_df2,POS_df2,Haplotype_mapped
0,chr2,233763993,rs6742078,G,T,"ADP=27;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.6524,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:152:27:27:0:27:100%:5.1363E-16:0:62:0:0:23:4,Covered,UGT1A1,chr2,233760270.0,233773300.0,Snp,3495.0,chr2,233760498.0,UGT1A1*6
1,chr7,99648291,rs15524,A,G,"ADP=16;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.652,0.3...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:63:57:7:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,-16921.0,chr7,99665212.0,CYP3A5*6
2,chr7,99668695,rs4646450,G,A,"ADP=18;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5329,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:32:18:18:9:9:50%:5.1645E-4:69:58:9:0:7:2,Covered,CYP3A5,chr7,99648194.0,99679996.0,Snp,3483.0,chr7,99665212.0,CYP3A5*6
3,chr7,99763843,rs2242480,C,T,"ADP=45;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5783,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:85:45:45:22:23:51.11%:2.6125E-9:60:55:18:4...,Covered,CYP3A4,chr7,99756967.0,99784184.0,Snp,-160.0,chr7,99764003.0,CYP3A4*18
4,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*4,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
5,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*6A,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
6,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*6J,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
7,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*6O,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
8,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*7B,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
9,chr8,18400285,rs1041983,C,T,"ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.6026,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:77:34:34:14:20:58.82%:1.6951E-8:58:56:13:1...,Covered,NAT2,chr8,18391282.0,18401218.0,NAT2*7G,NaN,chr8,18400285.0,"NAT2*6O,NAT2*7G,NAT2*7B,NAT2*6A,NAT2*13A,NAT2*..."
